# Machine Learning Workflow: Data Processing & Model Training

Any given color image can be considered a matrix of pixels where each pixel contains (R,G,B) values between 0-255. 

Each color is represented as an unsigned 8-bit integer. Grayscale images are represented by one byte per pixel. In this lab, we will learn how to read grayscale images into an ndarray or data frame and manipulate them to do some interesting operations on the image collection. We will use kmeans to cluster a set of images and understand how well they are being clustered into correct segments using k-means. 

### Clustering Images
Given a repository of images of handwritten digits, we can use unsupervised learning methods such as k-means to group the images into k clusters. Ideally with k=10, we should be able to group all images of 0,1,...,9 into 10 clusters. However, in practice this may not be possible as per reasons discussed in the lecture. In this part of the lab, we will look for the minimum k, that will provide us with the representation of all digits from 0 to 9. For the image data set, we will use the 1700+ low resolution 8x8 handwritten digit images from sklearn and apply k-means algorithm to find how well it can group/cluster into similar digits. 

In [ ]:
# !pip install --user scikit-learn

### Loading Defaults
Load the digits from sklearn into an image array. Print the first 5 images. 

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)
print(digits.target)   # actual labels of the images
import matplotlib.pyplot as plt
plt.gray()
for k in range(5):
    #print(digits.target[k])
    plt.matshow(digits.images[k])
plt.show()
print(digits.target[0])

### Flatten the 2D images
The original images are 8x8 matrices of integers who values are between 0-255. Flatten them to represent images as dimension 64 vectors. We will use these vectors in the kmeans algorithm as our data vectors. Store each image as a row in the matrix called flatten_images. The flatten_images should be dimension n x 64 where n is the number of images downloded. The shape of the flatten images should be (1797, 64)

In [ ]:
import numpy as np


from sklearn.datasets import load_digits
digits = load_digits()
flatten_images = digits.images.reshape((len(digits.images), -1))

print(flatten_images.shape)

## Implementing k-means
We note that, as the images are labeled, any supervised learning algorithm can be applied to this collection of images. However, our objective is to see if we can use unsupervided learning methods such as k-means to cluster images (w/o considering their labels). 

### kmeans++
The original k-means algorithm choose cluster centers at random. However, kmeans++ allows the selection of initial centers away from each other to avoid cluster centers converging together and loss converging to a local minima. 
The algorithm for kmeans++ (selecting initial cluster centers) is given by

<img src="kmeans-pp.jpg" alt="Alt text" height="100" width="300">

GIven below is a sample implementation of kmeans++.

In [ ]:
import numpy as np

def kmeans_plus(X, k):
    # Select the first cluster center randomly
    centers = [X[np.random.choice(X.shape[0], 1, replace=False)][0]]

    # Select the remaining cluster centers using k-means++
    for i in range(1, k):
        # Compute the distances to the nearest cluster center for each data point
        distances = np.min(np.sum((X - centers[-1])**2, axis=1)[:, np.newaxis], axis=1)

        # Compute the probabilities of each data point being selected as the next cluster center
        probs = distances / np.sum(distances)

        # Select the next cluster center randomly, weighted by the probabilities
        centers.append(X[np.random.choice(X.shape[0], 1, replace=False, p=probs)][0])

    return np.array(centers)
Mu = kmeans_plus(flatten_images, 3)   
Mu   # print the initial centers with k = 3

> The complexity of the kmeans++ code above is O(k\*n\*m). The complexity can be found by first  looping through the additional clusters k-1 times (aside from the first one, which is O(1)). The complexity is thus O(n) since the distances are being calculated in an n-dimensional space. Over m data points, the complexity becomes O(n\*m).  O(m) also represents the sum of the distances. 

> O(1) + (k-1) * \[(O(m\*n) + O(m) + O(1)\] = O(k\*n\*m)

### Is kmeans++ always better than random? 
In original k-means algorithm, we choose initial centers at random. This can lead to centers that will make error converging to local minimas. Here, we are exploring if the idea of using kmeans++ is a better way to choose the initial centers than choosing them at random.

We will test this with the 2D data set  [1, 1], [0, 1], [1, 0], [2, 1], [1, 2], [3, 2], [2, 3], [4, 10], [10, 4], [5, 5] of 10 points. Define the error as the sum of the manhattan distances of each point from the "closest' centers. Run the random algorithm t iterations (t=10, 15, 25, 50, 100) and record total errors in each case. For each t, print the probability that k-means++ choose centers better than the random algorithm? It is possible that random selection can sometimes work better than kmeans++. Run a few times to form an opinion.

In [ ]:
# manhattan distance for 2D vectors
def manhattan(x, y):
    return (abs(x[0]-y[0]) + abs(x[1]-y[1]))

In [ ]:
import numpy as np
x = np.array([[1, 1], [0, 1], [1, 0], [2, 1], [1, 2], [3, 2], [2, 3], [4, 10], [10, 4], [5, 5]])
n = 10   # num points
list_iter = [10, 15, 20, 25, 50, 100]   # num iterations list
k = 2 # given



def rand(x, k):
    indices = np.random.choice(x.shape[0], k, replace=False)
    return x[indices]

def errors(x, centers):
    error = 0

    for point in x:
        distances = [manhattan(point, center) for center in centers]
        error += min(distances)
    return error

for i in list_iter:
    kmeans_pro = 0
    
    for j in range(i):
        k_center = kmeans_plus(x, k)
        r_center = rand(x, k)
        
        k_error = errors(x, k_center)
        r_error = errors(x, r_center)
        
        if k_error < r_error:
            kmeans_pro += 1
            
    c = kmeans_pro / i
    
    print(f"Over {i} iterations there is a {c} chance that k-means++ beats random.")

> Thus, on average, k-means++ is better than random variable creation for selecting initial cluster centers. Thus, the clustering is more precise, and using k-means++ to choose centers is a better idea.

### Implementing k-means algorithm using Matrix forms

Max iterations of 100 or until the difference between prev_error and current_error is less than $10^{-5}$ whichever comes first. The function will return the k-centers (Mu) and loss. Loss generally will go down as you increase k.


Running the kmeans algorithm with random initial points:

In [ ]:
import numpy as np
k = 10


def kmeans_matrix(x, k):
    rows, cols = x.shape
    mu = x[np.random.choice(rows, k, replace=False)]

    for i in range(100):
        d = np.sqrt(((x[:, np.newaxis, :] - mu[np.newaxis, :, :]) ** 2).sum(axis=2)) # find euclidean distance to all centers
        cluster = np.argmin(d, axis=1)
        temp = np.array([x[cluster == j].mean(axis=0) for j in range(k)]) #calculate clusters
        
        if np.allclose(mu, temp): #break if converging
            break

        mu = temp
        
    loss = np.sum((x - mu[cluster]) ** 2)
    return mu, loss

for i in range(10, 31):
    mu, loss = kmeans_matrix(flatten_images, i)
    print('k=', i, ' Loss:', loss)

Running the kmeans algorithm with kmeans++ initial points:

In [ ]:
import numpy as np
k = 10


def kmeans_plus(x, k):
    centers = [x[np.random.choice(x.shape[0], 1, replace=False)][0]] # random center
    
    for i in range(1, k):
        dist = np.min(np.array([np.sum((x - center) ** 2, axis=1) for center in centers]), axis=0) # min distance to centers
        centers.append(x[np.random.choice(x.shape[0], 1, replace=False, p=dist / np.sum(dist))][0]) # select and add new center
    return np.array(centers)

def kmeans_matrix(x, k):
    rows, cols = x.shape
    mu = kmeans_plus(x, k)

    for iter in range(100):
        d = np.sqrt(((x[:, np.newaxis, :] - mu[np.newaxis, :, :]) ** 2).sum(axis=2))
        cluster = np.argmin(d, axis=1)
        temp = np.array([x[cluster == j].mean(axis=0) for j in range(k)])
        if np.allclose(mu, temp):
            break
        mu = temp
        
    loss = np.sum((x - mu[cluster]) ** 2)
    return mu, loss

for i in range(10,31):
    mu, loss = kmeans_matrix(flatten_images, i)
    print('k=', i, ' Loss:', loss)

> Overall, it is seen that k_means++ has a lower average loss across different values of k. This shows that k_means++ is better for starting the algorithm, which could mean more optimized clustering.

> Also, as k increases, the loss decreases since having more clusters means the data's structure is used more accurately. However, the decrease is not consistent, and sometimes the loss does increase as k increases. This means that there is a point where adding more clusters does not result in better perforamnce. 

### Alternate implementation of k-means
Given below is an alternate implementation of k-means to matrix implementation.

In [ ]:
import numpy as np

def kmeans_alt(X, k, max_iter=100):
    
    # randomly initialize k centroids
    Mu = X[np.random.choice(X.shape[0], size=k, replace=False)]
    # choose initial means using kmeans++
    #Mu = kmeans_pp(flatten_images, k)
    prev_loss = np.inf
    for i in range(max_iter):
        # assign each data point to the nearest centroid
        distances = np.linalg.norm(X[:, np.newaxis, :] - Mu, axis=2)
        cluster_assignment = np.argmin(distances, axis=1)

        # compute loss function
        loss = np.sum(np.square(np.min(distances, axis=1)))
        
        # recompute centroids as the mean of the assigned data points
        for j in range(k):
            Mu[j] = np.mean(X[cluster_assignment == j], axis=0)

        # check for convergence
        if np.abs(prev_loss - loss) < 1e-5:
            break
        prev_loss = loss
    return cluster_assignment, Mu, loss

for i in range(10,31):
    cluster_assignment, Mu, loss = kmeans_alt(flatten_images, i)
    print('k=', i, ' ', loss)

## Images in a Cluster

All images in the collection will be placed into one of the k clusters. We need to find all images in the cluster. We should see that the majority of the images in a cluster are quite similar; we can verify this by printing the dominant image in the cluster. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from collections import Counter
j = 12   # cluster number



k = 15
mu, _ = kmeans_matrix(flatten_images, k)

d = np.sqrt(((flatten_images[:, np.newaxis, :] - mu[np.newaxis, :, :]) ** 2).sum(axis=2))
cluster = np.argmin(d, axis=1)
i = np.where(cluster == j)[0]
centroid = np.sqrt(np.sum((flatten_images[i] - mu[j]) ** 2, axis=1))

print("Cluster image labels:", j, digits.target[i])
plt.imshow(digits.images[i[np.argmin(centroid)]])
plt.title(f"Cluster {j} (Index: {i[np.argmin(centroid)]})")

plt.show() # dominant image

fig, axes = plt.subplots(1, min(len(i), 10), figsize=(10, 3))
for ax, image, label in zip(axes, digits.images[i], digits.target[i]):
    ax.set_axis_off()
    ax.imshow(image)
    ax.set_title(label)
plt.show() # images in the cluster

### Error rate
Here, we will actually use the original labels of the images to see error rate in a specific cluster. We define the error rate as the ratio of **(all digits - number_of_dominant_digit_in_the cluster)/ all digits** in the cluster. For example, if the cluter had a total of 100 elements and 80 of them were the same digit (say 2), the the error rate is (100-80)/100 = 0.2. 

In [ ]:


k = 20
Mu, _ = kmeans_matrix(flatten_images, k)
d = np.sqrt(((flatten_images[:, np.newaxis, :] - Mu[np.newaxis, :, :]) ** 2).sum(axis=2))
cluster = np.argmin(d, axis=1)

for j in range(min(k, 20)):
    label = digits.target[np.where(cluster == j)[0]]
    most_common_digit, n = Counter(label).most_common(1)[0] # n = most common cluster
    error = (len(label) - n) / len(label) # (all digits - number_of_dominant_digit_in_the cluster) / all digits

    print(f"Cluster {j} - Dominant Digit = {most_common_digit}, Error = {error:.2f}")

### Best predicted Clusters
If the error rate of a specific cluster is almost zero, then they are the best predicted clusters. 


> The best predicted clusters are the ones with a near-zero error rate, e.g. closers 0, 3, 9, 13, 15, 18. 

## Number of Clusters
One of the most challenging part of clustering is to determine the minimum number of clusters required to solve the problem. In this case, we know, ideally we need 10 clusters. But with k=10, we may not get a unique dominant integer in a each cluster. So In this part, we will explore ways to determine the minimum value of k, that can give us k clusters that contains all digits 0..9. 

### Finding a minimum k 
It may be the case that if we go with k=10, we may not find that 10 cluster centers that do represent the digits from 0...9. Instead we may have to increase k, in order to find a k where each center is now representing one of 0...9. Here, we need to find the minimum k, where each cluster will be dominated by one of 0..9.

In [ ]:

def first_index_with_all_integers(arr):
    
    for k in range(10, 21):
        mu, _ = kmeans_matrix(arr, k)
        d = np.sqrt(((arr[:, np.newaxis, :] - mu[np.newaxis, :, :]) ** 2).sum(axis=2))
        cluster = np.argmin(d, axis=1)
        represented = set()
        
        for j in range(k):
            i = np.where(cluster == j)[0]
            if len(i) == 0:
                continue
            res, _ = Counter(digits.target[i]).most_common(1)[0] # res = most common digit
            represented.add(res)

        if len(represented) == 10:
            return k 

    return None

print("Minimum k:", first_index_with_all_integers(flatten_images))

Using the minimum k found above, printed below are the cluster centers as 8x8 images.

In [ ]:

k = 14
mu, _ = kmeans_matrix(flatten_images, k)

fig, axes = plt.subplots(1, k, figsize=(k, 1.5))
for i, ax in enumerate(axes):
    ax.imshow(Mu[i].reshape(8, 8))
    ax.axis('off')
    ax.set_title(f'cluster {i}')

plt.tight_layout()
plt.show()

> since there are only 14 clusters, some digits are combined into the same cluster.

### Finding a good k by plotting
Finding a k is often context dependent. Ideally, in this case, we would like to group handwritten digits into 10 clusters. However, we may need more than 10 clusters to capture all digits. So, we will plot the k vs loss to see if we can get some idea about the minimum k. 

### Finding vectors loss and k

In [ ]:
import numpy as np
x=[]
y=[]
print('k            loss')
for i in range(10,30):
    cluster_assignment, mu, loss = kmeans_alt(flatten_images, i)
    x.append(i)
    y.append(loss)
    print('k=', i, ' ', loss)

### Plot Loss vs k

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# use x and y vectors from previous part
x = x
y = y



plt.figure(figsize=(10, 6))
plt.plot(x, y, color='blue', label='Datapoints') 
plt.xlabel('Clusters')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.show()

### Finding the optimal k-value

> Just based on the loss, a k-value of around 29 is ideal. The loss begins to reach a limit, showing that it is not worth reducing the loss by using additional clusters.

## Timing kmeans

### Convergence and timing kmeans with matrix implementation.

In [ ]:
import time


res = []

for k in range(10, 31):
    start_time = time.time()
    mu, loss = kmeans_matrix(flatten_images, k)
    end_time = time.time()
    res.append((k, round(loss), end_time - start_time))
    
for result in res:
    print("When k = {:2}, loss ={:10} and time elapsed = {:.6f}s".format(*result))

### Convergence and Timing kmeans with kmeans_alt

In [ ]:
import time


res2 = []

for k in range(10, 31):
    start_time = time.time()
    cluster_assignment, mu, loss = kmeans_alt(flatten_images, k)
    end_time = time.time()
    res2.append((k, round(loss) , end_time - start_time))

for result in res2:
    print("When k = {:2}, loss ={:10} and time elapsed = {:.6f}s".format(*result))

### Plotting data size vs time:

In [ ]:
import random
import math
import time
k=18
m = []
t = []
m_values = range(100, 1100, 100)
times = []



for m in m_values:
    i = random.sample(range(len(flatten_images)), m)
    start_time = time.time()
    mu, loss = kmeans_matrix(flatten_images[i], k)
    end_time = time.time()
    times.append(end_time - start_time)

print("Data Size:", list(m_values))
print("Time elapsed:", times)
# print(m)
# print(t)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# use m and t vectors from previous part
x = m
y = t



import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(m_values, times)
plt.xlabel('Data Size')
plt.ylabel('Time (s)')
plt.grid(True)

plt.show()

> The graph shows that there is a slightly linear increase in the amount of time it takes to run the k-means algorithm. Every iteration of the algorithm, the distance between datapoints and clusters is calculated and the time complexity increases linearly. This confirms the complexity analysis, O(m\*n\*t\*k), of 2.1.